We periodically cache information about taxa of interest from the NatureServe species service in the Taxonomic Information Registry. This information is displayed in places like the State Wildlife Action Plan (SWAP) online app and the National Biogeographic Map.

NatureServe data are stored as key/value pairs in an hstore column of the "tir2" data store in the experimental GC2 platform we are working with. This notebook shows a couple of methods for retrieving NatureServe information via the GC2 API for use in web applications.

There is a NatureServe record for every taxon that is registered in the TIR. When no information from NatureServe was available with the query (based on scientific name lookup of the Element Global ID and retrieval based on that ID), there will be a status=>Not Found (or status=>error in cases where the code ran into a problem with the NatureServe service) key/value and a cacheDate, incidating the date/time the query was run.

The following queries, based on the SQL API, are probably the best options with hstore values. The data have also been piped to ElasticSearch in the GC2 instance, but they will come back as strings that need to be parsed instead of simple JSON. Note that there should only be a single result coming back for any given TSN or any originally submitted scientific name from the SGCN. However, there is not an absolute requirement that only a single record exist in the database for any of these key identifying features. You might harden your code by checking to make sure that there is only a single record and dealing with any duplicates.

In [2]:
import requests
from IPython.display import display

In [3]:
# Query based on ITIS tsn
tsn = 201922

q = "SELECT to_json(natureserve) as natureserve FROM tir.tir2 \
    WHERE itis->'discoveredTSN' = '"+str(tsn)+"' \
    OR itis->'acceptedTSN' = '"+str(tsn)+"'"

r = requests.get("https://gc2.mapcentia.com/api/v1/sql/bcb?q="+q).json()

display (r["features"][0]["properties"]["natureserve"])

{'GlobalStatusRank': 'G3T1',
 'StateCode:Nevada': 'S1',
 'cacheDate': '2017-05-07T18:16:00.816966',
 'elementGlobalID': 'ELEMENT_GLOBAL.2.102241',
 'globalStatusLastReviewed': '2001-07-02',
 'roundedGlobalStatusRankDescription': 'Critically Imperiled',
 'usNationalStatusLastReviewed': '2001-07-02',
 'usNationalStatusRankCode': 'N1'}

In [4]:
# Query based on a submitted SGCN scientific name
scientificname = "Hesperia dacotae"

q = "SELECT to_json(natureserve) as natureserve FROM tir.tir2 \
    WHERE registration->'SGCN_ScientificName_Submitted' = '"+scientificname+"'"

r = requests.get("https://gc2.mapcentia.com/api/v1/sql/bcb?q="+q).json()

display (r["features"][0]["properties"]["natureserve"])

{'GlobalStatusRank': 'G2',
 'StateCode:Illinois': 'SX',
 'StateCode:Iowa': 'S1',
 'StateCode:Minnesota': 'S2',
 'StateCode:North Dakota': 'S2',
 'StateCode:South Dakota': 'S2',
 'cacheDate': '2017-05-07T18:17:19.612952',
 'elementGlobalID': 'ELEMENT_GLOBAL.2.119247',
 'globalStatusLastReviewed': '2011-03-22',
 'roundedGlobalStatusRankDescription': 'Imperiled',
 'usNationalStatusLastReviewed': '2003-04-08',
 'usNationalStatusRankCode': 'N2'}

In [5]:
# Query based on a known NatureServe ID
elementGlobalID = "ELEMENT_GLOBAL.2.140209"

q = "SELECT to_json(natureserve) as natureserve FROM tir.tir2 \
    WHERE natureserve->'elementGlobalID' = '"+elementGlobalID+"'"

r = requests.get("https://gc2.mapcentia.com/api/v1/sql/bcb?q="+q).json()

display (r["features"][0]["properties"]["natureserve"])

{'GlobalStatusRank': 'G5',
 'StateCode:Alaska': 'S1',
 'StateCode:New York': 'S2',
 'StateCode:Vermont': 'S1',
 'cacheDate': '2017-05-07T18:17:08.592186',
 'elementGlobalID': 'ELEMENT_GLOBAL.2.140209',
 'globalStatusLastReviewed': '2015-08-04',
 'roundedGlobalStatusRankDescription': 'Secure',
 'usNationalStatusLastReviewed': 'Unknown',
 'usNationalStatusRankCode': 'NNR'}